In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 128
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy = 1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import TripletNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1024,2048)
triplet_network = TripletNetwork(attribute_network, metric_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=200000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024

        #print(batch_features_ext)

        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
print(triplet_network)

TripletNetwork(
  (embeddingnet_att): AttributeNetwork(
    (fc1): Linear(in_features=85, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=2048, bias=True)
  )
  (embeddingnet): MetricNetwork(
    (fc1): Linear(in_features=2048, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=2048, bias=True)
  )
)


In [8]:
print("training...")
last_accuracy = 0.0
writer = SummaryWriter()
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=batch_labels.numpy()[i])[0], replace=True)
        batch_features_negative[i] = batch_features[pick]    

    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss)
        writer.add_scalar('data/loss', loss, episode)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)
        writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        writer.add_scalar('data/H', H, episode)
writer.export_scalars_to_json("./test.json")
writer.close()


training...
episode: 1000 loss tensor(1.00000e-02 *
       5.7284, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.423570800351803
gzsl: unseen=0.2310 , seen=0.5948 , h=0.3327
____________________________________________________________________________________________________
episode: 2000 loss tensor(1.00000e-02 *
       4.4071, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.44450307827616536
gzsl: unseen=0.2262 , seen=0.6509 , h=0.3357
____________________________________________________________________________________________________
episode: 3000 loss tensor(1.00000e-02 *
       5.2074, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4914687774846086
gzsl: unseen=0.2433 , seen=0.6843 , h=0.3589
____________________________________________________________________________________________________
episode: 4000 loss tensor(1.00000e-02 *
       5.5884, device='cuda:0')
Testing...
class num: 10
class 

episode: 29000 loss tensor(1.00000e-03 *
       7.3409, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5572559366754617
gzsl: unseen=0.2716 , seen=0.7900 , h=0.4042
____________________________________________________________________________________________________
episode: 30000 loss tensor(1.00000e-03 *
       7.0689, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5525065963060686
gzsl: unseen=0.2744 , seen=0.8019 , h=0.4089
____________________________________________________________________________________________________
episode: 31000 loss tensor(1.00000e-02 *
       1.6800, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5621811785400176
gzsl: unseen=0.2872 , seen=0.7884 , h=0.4211
____________________________________________________________________________________________________
episode: 32000 loss tensor(1.00000e-02 *
       1.2488, device='cuda:0')
Testing...
class num: 10
class num: 50


episode: 57000 loss tensor(1.00000e-03 *
       3.2664, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5838170624450307
gzsl: unseen=0.3219 , seen=0.8088 , h=0.4605
____________________________________________________________________________________________________
episode: 58000 loss tensor(1.00000e-03 *
       6.0496, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5815303430079156
gzsl: unseen=0.3420 , seen=0.8038 , h=0.4798
____________________________________________________________________________________________________
episode: 59000 loss tensor(1.00000e-03 *
       3.3158, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5792436235708004
gzsl: unseen=0.3363 , seen=0.8078 , h=0.4749
____________________________________________________________________________________________________
episode: 60000 loss tensor(1.00000e-03 *
       9.3097, device='cuda:0')
Testing...
class num: 10
class num: 50


episode: 85000 loss tensor(1.00000e-03 *
       2.8790, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6228671943711521
gzsl: unseen=0.3770 , seen=0.8136 , h=0.5152
____________________________________________________________________________________________________
episode: 86000 loss tensor(1.00000e-03 *
       4.3587, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6165347405452947
gzsl: unseen=0.3339 , seen=0.8187 , h=0.4743
____________________________________________________________________________________________________
episode: 87000 loss tensor(1.00000e-03 *
       6.5777, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6302550571679859
gzsl: unseen=0.3717 , seen=0.8157 , h=0.5107
____________________________________________________________________________________________________
episode: 88000 loss tensor(1.00000e-03 *
       2.8294, device='cuda:0')
Testing...
class num: 10
class num: 50


episode: 113000 loss tensor(1.00000e-03 *
       2.9901, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5956024626209323
gzsl: unseen=0.3655 , seen=0.8060 , h=0.5029
____________________________________________________________________________________________________
episode: 114000 loss tensor(1.00000e-03 *
       2.7359, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6117854001759016
gzsl: unseen=0.3706 , seen=0.8144 , h=0.5094
____________________________________________________________________________________________________
episode: 115000 loss tensor(1.00000e-03 *
       2.6197, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6323658751099385
gzsl: unseen=0.3727 , seen=0.8233 , h=0.5132
____________________________________________________________________________________________________
episode: 116000 loss tensor(1.00000e-03 *
       2.6168, device='cuda:0')
Testing...
class num: 10
class num:

...

episode: 365000 loss tensor(1.00000e-03 *
       1.8209, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6058047493403694
gzsl: unseen=0.3546 , seen=0.8407 , h=0.4988
____________________________________________________________________________________________________
episode: 366000 loss tensor(1.00000e-03 *
       1.8257, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6145998240985049
gzsl: unseen=0.3873 , seen=0.8378 , h=0.5298
____________________________________________________________________________________________________
episode: 367000 loss tensor(1.00000e-03 *
       7.2629, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6093227792436235
gzsl: unseen=0.3868 , seen=0.8298 , h=0.5276
____________________________________________________________________________________________________
episode: 368000 loss tensor(1.00000e-03 *
       7.7151, device='cuda:0')
Testing...
class num: 10
class num:

episode: 393000 loss tensor(1.00000e-03 *
       2.2589, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.618117854001759
gzsl: unseen=0.4255 , seen=0.8328 , h=0.5632
____________________________________________________________________________________________________
episode: 394000 loss tensor(1.00000e-03 *
       1.7743, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.612664907651715
gzsl: unseen=0.4310 , seen=0.8320 , h=0.5678
____________________________________________________________________________________________________
episode: 395000 loss tensor(1.00000e-03 *
       1.7743, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6086191732629728
gzsl: unseen=0.3884 , seen=0.8386 , h=0.5309
____________________________________________________________________________________________________
episode: 396000 loss tensor(1.00000e-03 *
       1.7632, device='cuda:0')
Testing...
class num: 10
class num: 5

episode: 421000 loss tensor(1.00000e-03 *
       1.7430, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6153034300791557
gzsl: unseen=0.3989 , seen=0.8358 , h=0.5401
____________________________________________________________________________________________________
episode: 422000 loss tensor(1.00000e-03 *
       1.7695, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6156552330694811
gzsl: unseen=0.3945 , seen=0.8395 , h=0.5368
____________________________________________________________________________________________________
episode: 423000 loss tensor(1.00000e-03 *
       1.7569, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6056288478452067
gzsl: unseen=0.3836 , seen=0.8336 , h=0.5255
____________________________________________________________________________________________________
episode: 424000 loss tensor(1.00000e-03 *
       1.7579, device='cuda:0')
Testing...
class num: 10
class num:

episode: 449000 loss tensor(1.00000e-03 *
       4.4183, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6237467018469657
gzsl: unseen=0.4245 , seen=0.8384 , h=0.5636
____________________________________________________________________________________________________
episode: 450000 loss tensor(1.00000e-03 *
       1.7745, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6198768689533861
gzsl: unseen=0.4106 , seen=0.8399 , h=0.5515
____________________________________________________________________________________________________
episode: 451000 loss tensor(1.00000e-03 *
       1.7591, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6263852242744063
gzsl: unseen=0.4269 , seen=0.8374 , h=0.5655
____________________________________________________________________________________________________
episode: 452000 loss tensor(1.00000e-03 *
       2.3394, device='cuda:0')
Testing...
class num: 10
class num:

episode: 477000 loss tensor(1.00000e-03 *
       5.0970, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.61688654353562
gzsl: unseen=0.4091 , seen=0.8334 , h=0.5488
____________________________________________________________________________________________________
episode: 478000 loss tensor(1.00000e-03 *
       2.5210, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6010554089709762
gzsl: unseen=0.3843 , seen=0.8334 , h=0.5261
____________________________________________________________________________________________________
episode: 479000 loss tensor(1.00000e-03 *
       9.8326, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6094986807387863
gzsl: unseen=0.3989 , seen=0.8380 , h=0.5406
____________________________________________________________________________________________________
episode: 480000 loss tensor(1.00000e-03 *
       1.7100, device='cuda:0')
Testing...
class num: 10
class num: 5